<a href="https://colab.research.google.com/github/Ava-creates/Ava-creates.github.io/blob/main/linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Explore and import Boston house prices dataset

In [ ]:
#importing campus
import warnings
import numpy as np
import pandas as pd

from sklearn.datasets import load_boston
with warnings.catch_warnings():
# You should probably not use this dataset. LOVE THE WARNING
  warnings.filterwarnings("ignore")
  X, y = load_boston(return_X_y=True)

#df = pd.DataFrame(X)

print(y.shape)



(506,)


In [ ]:
#y.reshape((253,2))


#no idea why we are doing this tho


In [ ]:
'''
o=np.ones(X.shape[0])
X= np.insert(X, 0, o, axis=1)
print(X)

'''

'\no=np.ones(X.shape[0])\nX= np.insert(X, 0, o, axis=1)\nprint(X)\n\n'

#getting w wihtout ridge

In [ ]:
def get_weight(X, y ):
  #w=inv(xt x)xt y
  XT=np.transpose(X)
  w= np.array((np.linalg.inv(XT@X)) @ XT @y)
#linalg.inv(a)
  #print(w)
  return w

w=get_weight(X,y)
w.shape

(13,)

#getting w with ridge

In [ ]:
def get_weightridge(X, y , alpha):
  #w=inv(xt x)xt y
  XT=np.transpose(X)
  I= np.identity(X.shape[1], dtype="int")
  w= (np.linalg.inv((XT@X)+alpha*I)) @ XT @y
#linalg.inv(a)
  #print(w)
  #w[0] =  np.mean(y)
  #print(w)
  return w

get_weightridge(X,y, .001)

array([-9.28962007e-02,  4.87153511e-02, -4.06707120e-03,  2.85388960e+00,
       -2.86698669e+00,  5.92806695e+00, -7.27042723e-03, -9.68501856e-01,
        1.71151992e-01, -9.39650991e-03, -3.92188246e-01,  1.49055445e-02,
       -4.16312330e-01])

#hypothesis actual prediction happening

In [ ]:
def hypothesis(X, w):
  
  
  h=w@np.transpose(X) # h(w) 

  #print(h)
  #print(h.shape)
 # h=np.sum(h)
  return h

#hypothesis(X,w) #

#cost/loss function

In [ ]:
def eval_error(X_train, y_train, X_test, y_test, alpha):
  predict_train= hypothesis(X_train, get_weightridge(X_train, y_train, alpha ))  + np.mean(y_train)
  predict_test=hypothesis(X_test, get_weightridge(X_test, y_test, alpha)) + np.mean(y_train)

  t=(y_train-predict_train)**2
  
  train_err=np.mean(t)
  test_err=np.mean((y_test-predict_test)**2)

  #print(train_err)
  #print(test_err)
  
  return (train_err, test_err)




  


#k-folding and then doing the thing for all the 10 folds

In [ ]:
from sklearn.model_selection import KFold
def k_fold_regression(k, X, y, alpha):
  kf=KFold(n_splits=k, random_state=21, shuffle=True )

  d=0
  c=0
  a=[]
  for train_index, test_index in kf.split(X):
    X_train, X_test=X[train_index], X[test_index]
    y_train, y_test=y[train_index], y[test_index]
    
    #centering data to avoid getting b
    
    y_mean=np.mean(y_train)
    y_train-=y_mean
    y_test-=y_mean
    #scaling??

    #scaler= preproccessing_StandardScaler().fit()
    from sklearn import preprocessing 
    scaler_train = preprocessing.StandardScaler().fit(X_train)
    scaler_test = preprocessing.StandardScaler().fit(X_test)
    X_train_scaled = scaler_train.transform(X_train)
    X_test_scaled= scaler_test.transform(X_test)  
    a.append(eval_error(X_train_scaled, y_train, X_test_scaled, y_test, alpha))
   # print(eval_error(X_train_scaled, y_train, X_test_scaled, y_test, alpha))
  test_mean=0
  train_mean=0
  for i in a:
    test_mean+=i[1]
    train_mean+=i[0]

  return (train_mean/k, test_mean/k)



#k_fold_regression(10, X, y, .0001)








  

#finding the best alpha

In [ ]:
###trying linear regression with no ridge is better than ridge 10###
a,b=k_fold_regression(10, X, y, 0)
print("--------------------------------------")
print("train: ", a ,"\t", "test:", b )
print("\n")

--------------------------------------
train:  21.806183575851065 	 test: 16.460424621999827




In [ ]:
n = np.logspace(1, 7, num=13) 


In [ ]:

for i in n:
  print(i)
  a,b=k_fold_regression(10, X, y, i)
  print("--------------------------------------")
  print("train: ", a ,"\t", "test:", b )
  print("\n")


#10 is the best fit as it has the lowest error


10.0
--------------------------------------
train:  21.892901157570186 	 test: 18.871365859469368


31.622776601683793
--------------------------------------
train:  22.285444055697496 	 test: 23.667676104913077


100.0
--------------------------------------
train:  23.725488384882073 	 test: 34.29926125871303


316.22776601683796
--------------------------------------
train:  28.166554098540104 	 test: 50.55992463602915


1000.0
--------------------------------------
train:  38.53214872734354 	 test: 67.2339588178375


3162.2776601683795
--------------------------------------
train:  54.007026499554456 	 test: 77.82893450066003


10000.0
--------------------------------------
train:  69.25978171325319 	 test: 82.38050719533354


31622.776601683792
--------------------------------------
train:  78.53074243629268 	 test: 83.9881811181876


100000.0
--------------------------------------
train:  82.40325012491365 	 test: 84.51553966791388


316227.7660168379
-----------------------------

#adding polynomial feature to the data better result for degree 2

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly=PolynomialFeatures(2)
X_poly=poly.fit_transform(X)

for i in n:
  print(i)
  a,b=k_fold_regression(10, X_poly, y, i)
  print("--------------------------------------")
  print("train: ", a ,"\t", "test:", b )
  print("\n")


#polynomial feature is better

  


10.0
--------------------------------------
train:  10.049055874118874 	 test: 9.885809229746553


31.622776601683793
--------------------------------------
train:  12.751706269046785 	 test: 13.325586401894473


100.0
--------------------------------------
train:  16.2226905932108 	 test: 18.271071274361038


316.22776601683796
--------------------------------------
train:  19.700253646409855 	 test: 27.2851822711348


1000.0
--------------------------------------
train:  24.287457983108876 	 test: 41.5810366942791


3162.2776601683795
--------------------------------------
train:  33.086668319850524 	 test: 58.34878157435058


10000.0
--------------------------------------
train:  46.76199935941408 	 test: 72.47260047971982


31622.776601683792
--------------------------------------
train:  62.00900191683604 	 test: 80.17338052568638


100000.0
--------------------------------------
train:  74.28758894586744 	 test: 83.2228109176057


316227.7660168379
-------------------------------

#the end 


*  alpha 10 gives the best result
*  polynomisal feature of degree 2 gives better result

